In [4]:
import pandas as pd
import numpy as np
import xgboost as xgb

# 1. Cargar y agregar datos por producto y mes
df = pd.read_csv("sell-in.txt", sep="\t")
df["periodo"] = pd.to_datetime(df["periodo"], format="%Y%m")
df = df.groupby(["product_id", "periodo"])["tn"].sum().reset_index()
df = df.sort_values(["product_id", "periodo"])

# 2. Crear features
def crear_features(data):
    data = data.copy()
    data["month"] = data["periodo"].dt.month
    for lag in range(1, 4):
        data[f"lag_{lag}"] = data.groupby("product_id")["tn"].shift(lag)
    return data

df_feat = crear_features(df)

# 3. Leer universo de productos
productos_objetivo = pd.read_csv("product_id_apredecir201912.txt")["product_id"].tolist()

# 4. Inicializar
resultados = []
entrenados = 0
fallbacks = 0

# 5. Loop por producto
for producto in productos_objetivo:
    data_prod = df_feat[df_feat["product_id"] == producto].copy()
    train = data_prod[data_prod["periodo"] <= "2019-12-01"]
    train = train.dropna(subset=["lag_1", "lag_2", "lag_3"])

    try:
        if train.shape[0] >= 2:
            X_train = train[["month", "lag_1", "lag_2", "lag_3"]]
            y_train = train["tn"]

            dtrain = xgb.DMatrix(X_train, label=y_train)

            params = {
                "objective": "reg:squarederror",
                "eval_metric": "mae",
                "max_depth": 5,
                "eta": 0.1,
                "subsample": 0.7,
                "colsample_bytree": 0.7,
                "min_child_weight": 10,
                "verbosity": 0,
                "seed": 42
            }

            model = xgb.train(params, dtrain, num_boost_round=100, verbose_eval=False)

            dic = data_prod.loc[data_prod["periodo"] == "2019-12-01", "tn"]
            nov = data_prod.loc[data_prod["periodo"] == "2019-11-01", "tn"]
            octu = data_prod.loc[data_prod["periodo"] == "2019-10-01", "tn"]

            if not dic.empty and not nov.empty and not octu.empty:
                X_pred = pd.DataFrame([[2, dic.values[0], nov.values[0], octu.values[0]]],
                                      columns=["month", "lag_1", "lag_2", "lag_3"])
                dpred = xgb.DMatrix(X_pred)
                pred = model.predict(dpred)[0]
                resultados.append({"product_id": producto, "tn_predicho": pred})
                entrenados += 1
                continue

    except Exception as e:
        print(f"⚠️ Fallo producto {producto}: {e}")

    # Fallback combinado
    data_hist = df[(df["product_id"] == producto) & (df["periodo"] < "2020-01-01")].copy()
    ultimos_12 = data_hist.sort_values("periodo").tail(12)
    prom_ultimos12 = ultimos_12["tn"].mean() if not ultimos_12.empty else 0.0
    febreros = data_hist[data_hist["periodo"].dt.month == 2]
    prom_febreros = febreros["tn"].mean() if not febreros.empty else 0.0
    pred = 0.6 * prom_ultimos12 + 0.4 * prom_febreros

    resultados.append({"product_id": producto, "tn_predicho": pred})
    fallbacks += 1

# 6. Exportar
df_resultado = pd.DataFrame(resultados)
df_resultado.to_csv("predicciones_febrero2020_xgb_780_combinado_AGREGADO.csv", index=False)

print("✅ Archivo generado: predicciones_febrero2020_xgb_780_combinado_AGREGADO.csv")
print(f"🟢 Modelos entrenados: {entrenados}")
print(f"🟡 Fallbacks aplicados: {fallbacks}")
print(f"🔢 Total productos: {len(df_resultado)}")


✅ Archivo generado: predicciones_febrero2020_xgb_780_combinado_AGREGADO.csv
🟢 Modelos entrenados: 771
🟡 Fallbacks aplicados: 9
🔢 Total productos: 780


In [5]:
import pandas as pd
import numpy as np
import xgboost as xgb

# 1. Cargar y agregar datos
df = pd.read_csv("sell-in.txt", sep="\t")
df["periodo"] = pd.to_datetime(df["periodo"], format="%Y%m")
df = df.groupby(["product_id", "periodo"])["tn"].sum().reset_index()
df = df.sort_values(["product_id", "periodo"])

# 2. Crear features
def crear_features(data):
    data = data.copy()
    data["month"] = data["periodo"].dt.month
    for lag in range(1, 13):
        data[f"lag_{lag}"] = data.groupby("product_id")["tn"].shift(lag)
    data["delta_1"] = data["lag_1"] - data["lag_2"]
    data["delta_2"] = data["lag_2"] - data["lag_3"]
    data["delta_3"] = data["lag_3"] - data["lag_4"]
    return data

df_feat = crear_features(df)

# 3. Leer productos a predecir
productos_objetivo = pd.read_csv("product_id_apredecir201912.txt")["product_id"].tolist()

# 4. Inicializar
resultados = []
entrenados = 0
fallbacks = 0

# 5. Loop por producto
for producto in productos_objetivo:
    data_prod = df_feat[df_feat["product_id"] == producto].copy()
    train = data_prod[data_prod["periodo"] <= "2019-12-01"]
    train = train.dropna(subset=[f"lag_{i}" for i in range(1, 13)] + ["delta_1", "delta_2", "delta_3"])

    try:
        if train.shape[0] >= 5:
            features = ["month"] + [f"lag_{i}" for i in range(1, 13)] + ["delta_1", "delta_2", "delta_3"]
            X_train = train[features]
            y_train = train["tn"]

            dtrain = xgb.DMatrix(X_train, label=y_train)

            params = {
                "objective": "reg:squarederror",
                "eval_metric": "mae",
                "max_depth": 5,
                "eta": 0.1,
                "subsample": 0.7,
                "colsample_bytree": 0.7,
                "min_child_weight": 10,
                "verbosity": 0,
                "seed": 42
            }

            model = xgb.train(params, dtrain, num_boost_round=100, verbose_eval=False)

            fila_pred = data_prod.loc[data_prod["periodo"] == "2019-12-01"]
            if not fila_pred.empty and not fila_pred[features].isnull().any(axis=1).all():
                X_pred = fila_pred[features].iloc[0:1]
                dpred = xgb.DMatrix(X_pred)
                pred = model.predict(dpred)[0]
                resultados.append({"product_id": producto, "tn_predicho": pred})
                entrenados += 1
                continue

    except Exception as e:
        print(f"⚠️ Fallo producto {producto}: {e}")

    # Fallback combinado
    data_hist = df[(df["product_id"] == producto) & (df["periodo"] < "2020-01-01")].copy()
    ultimos_12 = data_hist.sort_values("periodo").tail(12)
    prom_ultimos12 = ultimos_12["tn"].mean() if not ultimos_12.empty else 0.0
    febreros = data_hist[data_hist["periodo"].dt.month == 2]
    prom_febreros = febreros["tn"].mean() if not febreros.empty else 0.0
    pred = 0.6 * prom_ultimos12 + 0.4 * prom_febreros

    resultados.append({"product_id": producto, "tn_predicho": pred})
    fallbacks += 1

# 6. Exportar
df_resultado = pd.DataFrame(resultados)
df_resultado.to_csv("predicciones_febrero2020_xgb_lags12_deltas_combinado.csv", index=False)

print("✅ Archivo generado: predicciones_febrero2020_xgb_lags12_deltas_combinado.csv")
print(f"🟢 Modelos entrenados: {entrenados}")
print(f"🟡 Fallbacks aplicados: {fallbacks}")
print(f"🔢 Total productos: {len(df_resultado)}")


✅ Archivo generado: predicciones_febrero2020_xgb_lags12_deltas_combinado.csv
🟢 Modelos entrenados: 608
🟡 Fallbacks aplicados: 172
🔢 Total productos: 780


In [6]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings("ignore")

# 1. Cargar y agregar
df = pd.read_csv("sell-in.txt", sep="\t")
df["periodo"] = pd.to_datetime(df["periodo"], format="%Y%m")
df = df.groupby(["product_id", "periodo"])["tn"].sum().reset_index()
df = df.sort_values(["product_id", "periodo"])

# 2. Reemplazo de agosto
def reemplazar_agosto(grupo):
    grupo = grupo.copy()
    for i in range(len(grupo)):
        if grupo.iloc[i]["periodo"].month == 8 and i >= 12:
            grupo.iloc[i, grupo.columns.get_loc("tn")] = grupo.iloc[i-12:i]["tn"].mean()
    return grupo

df_corr = df.groupby("product_id").apply(reemplazar_agosto).reset_index(drop=True)

# 3. Features
def crear_features(data):
    data = data.copy()
    data["month"] = data["periodo"].dt.month
    for lag in range(1, 13):
        data[f"lag_{lag}"] = data.groupby("product_id")["tn"].shift(lag)
    data["delta_1"] = data["lag_1"] - data["lag_2"]
    data["delta_2"] = data["lag_2"] - data["lag_3"]
    data["delta_3"] = data["lag_3"] - data["lag_4"]
    return data

df_feat = crear_features(df_corr)

# 4. Leer universo
productos_objetivo = pd.read_csv("product_id_apredecir201912.txt")["product_id"].tolist()

# 5. Inicializar
resultados = []
entrenados = 0
fallbacks = 0

# 6. Loop por producto
for producto in productos_objetivo:
    data_prod = df_feat[df_feat["product_id"] == producto].copy()
    train = data_prod[data_prod["periodo"] <= "2019-12-01"]
    train = train.dropna(subset=[f"lag_{i}" for i in range(1, 13)] + ["delta_1", "delta_2", "delta_3"])

    try:
        if train.shape[0] >= 15:
            features = ["month"] + [f"lag_{i}" for i in range(1, 13)] + ["delta_1", "delta_2", "delta_3"]
            X = train[features]
            y = train["tn"]

            maes = []
            for train_idx, val_idx in KFold(n_splits=5, shuffle=False).split(X):
                X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
                y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

                modelo = xgb.XGBRegressor(
                    objective="reg:squarederror",
                    learning_rate=0.1,
                    n_estimators=100,
                    max_depth=6,
                    subsample=0.8,
                    colsample_bytree=0.8,
                    seed=42,
                    verbosity=0
                )
                modelo.fit(X_train, y_train)
                y_pred_val = modelo.predict(X_val)
                maes.append(mean_absolute_error(y_val, y_pred_val))

            # Entrenamiento final
            modelo.fit(X, y)

            fila_pred = data_prod.loc[data_prod["periodo"] == "2019-12-01"]
            if not fila_pred.empty and not fila_pred[features].isnull().any(axis=1).all():
                X_pred = fila_pred[features].iloc[0:1]
                pred = modelo.predict(X_pred)[0]
                resultados.append({"product_id": producto, "tn_predicho": pred})
                entrenados += 1
                continue

    except Exception as e:
        print(f"⚠️ Fallo producto {producto}: {e}")

    # Fallback
    data_hist = df[(df["product_id"] == producto) & (df["periodo"] < "2020-01-01")]
    ultimos_12 = data_hist.sort_values("periodo").tail(12)
    prom_ultimos12 = ultimos_12["tn"].mean() if not ultimos_12.empty else 0.0
    febreros = data_hist[data_hist["periodo"].dt.month == 2]
    prom_febreros = febreros["tn"].mean() if not febreros.empty else 0.0
    pred = 0.6 * prom_ultimos12 + 0.4 * prom_febreros

    resultados.append({"product_id": producto, "tn_predicho": pred})
    fallbacks += 1

# 7. Exportar
df_resultado = pd.DataFrame(resultados)
df_resultado.to_csv("predicciones_febrero2020_xgb_agosto_cv.csv", index=False)

print("✅ Archivo generado: predicciones_febrero2020_xgb_agosto_cv.csv")
print(f"🟢 Modelos entrenados con CV: {entrenados}")
print(f"🟡 Fallbacks aplicados: {fallbacks}")
print(f"🔢 Total productos: {len(df_resultado)}")


✅ Archivo generado: predicciones_febrero2020_xgb_agosto_cv.csv
🟢 Modelos entrenados con CV: 549
🟡 Fallbacks aplicados: 231
🔢 Total productos: 780


In [7]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings("ignore")

# 1. Cargar y agregar
df = pd.read_csv("sell-in.txt", sep="\t")
df["periodo"] = pd.to_datetime(df["periodo"], format="%Y%m")
df = df.groupby(["product_id", "periodo"])["tn"].sum().reset_index()
df = df.sort_values(["product_id", "periodo"])

# 2. Reemplazo de agosto
def reemplazar_agosto(grupo):
    grupo = grupo.copy()
    for i in range(len(grupo)):
        if grupo.iloc[i]["periodo"].month == 8 and i >= 12:
            grupo.iloc[i, grupo.columns.get_loc("tn")] = grupo.iloc[i-12:i]["tn"].mean()
    return grupo

df_corr = df.groupby("product_id").apply(reemplazar_agosto).reset_index(drop=True)

# 3. Features
def crear_features(data):
    data = data.copy()
    data["month"] = data["periodo"].dt.month
    for lag in range(1, 13):
        data[f"lag_{lag}"] = data.groupby("product_id")["tn"].shift(lag)
    data["delta_1"] = data["lag_1"] - data["lag_2"]
    data["delta_2"] = data["lag_2"] - data["lag_3"]
    data["delta_3"] = data["lag_3"] - data["lag_4"]
    return data

df_feat = crear_features(df_corr)

# 4. Leer universo
productos_objetivo = pd.read_csv("product_id_apredecir201912.txt")["product_id"].tolist()

# 5. Inicializar
resultados = []
maes_xgb = []
entrenados = 0
fallbacks = 0

# 6. Loop por producto
for producto in productos_objetivo:
    data_prod = df_feat[df_feat["product_id"] == producto].copy()
    train = data_prod[data_prod["periodo"] <= "2019-12-01"]
    train = train.dropna(subset=[f"lag_{i}" for i in range(1, 13)] + ["delta_1", "delta_2", "delta_3"])

    try:
        if train.shape[0] >= 15:
            features = ["month"] + [f"lag_{i}" for i in range(1, 13)] + ["delta_1", "delta_2", "delta_3"]
            X = train[features]
            y = train["tn"]

            maes = []
            for train_idx, val_idx in KFold(n_splits=5, shuffle=False).split(X):
                X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
                y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

                modelo = xgb.XGBRegressor(
                    learning_rate=0.1,
                    n_estimators=100,
                    max_depth=6,
                    subsample=0.8,
                    colsample_bytree=0.8,
                    random_state=42,
                    verbosity=0
                )
                modelo.fit(X_train, y_train)
                y_pred_val = modelo.predict(X_val)
                maes.append(mean_absolute_error(y_val, y_pred_val))

            mae_cv = np.mean(maes)
            modelo.fit(X, y)

            fila_pred = data_prod.loc[data_prod["periodo"] == "2019-12-01"]
            if not fila_pred.empty and not fila_pred[features].isnull().any(axis=1).all():
                X_pred = fila_pred[features].iloc[0:1]
                pred = modelo.predict(X_pred)[0]
                resultados.append({"product_id": producto, "tn_predicho": pred})
                maes_xgb.append({"product_id": producto, "mae_cv": mae_cv})
                entrenados += 1
                continue

    except Exception as e:
        print(f"Fallo producto {producto}: {e}")

    # Fallback
    data_hist = df[(df["product_id"] == producto) & (df["periodo"] < "2020-01-01")]
    ultimos_12 = data_hist.sort_values("periodo").tail(12)
    prom_ultimos12 = ultimos_12["tn"].mean() if not ultimos_12.empty else 0.0
    febreros = data_hist[data_hist["periodo"].dt.month == 2]
    prom_febreros = febreros["tn"].mean() if not febreros.empty else 0.0
    pred = 0.6 * prom_ultimos12 + 0.4 * prom_febreros

    resultados.append({"product_id": producto, "tn_predicho": pred})
    maes_xgb.append({"product_id": producto, "mae_cv": 9999})
    fallbacks += 1

# 7. Exportar
pd.DataFrame(resultados).to_csv("predicciones_febrero2020_xgb_agosto_cv1.csv", index=False)
pd.DataFrame(maes_xgb).to_csv("mae_xgb_cv1.csv", index=False)

print("Archivo generado: predicciones_febrero2020_xgb_agosto_cv1.csv")
print("MAE por producto: mae_xgb_cv1.csv")
print(f"Modelos entrenados: {entrenados}")
print(f"Fallbacks aplicados: {fallbacks}")
print(f"Total productos: {len(productos_objetivo)}")

Archivo generado: predicciones_febrero2020_xgb_agosto_cv1.csv
MAE por producto: mae_xgb_cv1.csv
Modelos entrenados: 549
Fallbacks aplicados: 231
Total productos: 780
